In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *


In [2]:
from quchem.LCU_method import *
from quchem.Unitary_partitioning import * 
from quchem.quantum_circuit_functions import *
from quchem.Ansatz_Generator_Functions import *

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end

#####################################

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:17<00:00, 35.66it/s] 


{0: [(-3.9344419567579045+0j) []],
 1: [(0.07823637778985218+0j) [Z6 Z7]],
 2: [(0.07823637778985211+0j) [Z8 Z9]],
 3: [(-0.21675429323818407+0j) [Z8],
  (-0.004217284878422725+0j) [Y6 Y7 X8 X9],
  (0.0030498840144275855+0j) [Y5 Z6 Z7 Y8 X9 X10],
  (0.004889538713703989+0j) [Y2 Z3 Z4 X5 X8 Y9],
  (0.002562390289419119+0j) [Y0 Z1 Z2 Z3 Z4 X5 X8 Y9],
  (0.010590593747758628+0j) [Y4 X5 X8 Y9]],
 4: [(-0.21675429323818407+0j) [Z7],
  (-0.004217284878422725+0j) [X6 X7 Y8 Y9],
  (0.0048895387137039914+0j) [X2 Z3 Z4 Y5 Y6 X7],
  (0.0030498840144275847+0j) [X5 X6 Y7 Z8 Z9 Y10],
  (0.00256239028941912+0j) [X0 Z1 Z2 Z3 Z4 Y5 Y6 X7],
  (0.01059059374775863+0j) [X4 Y5 Y6 X7]],
 5: [(-0.2167542932381842+0j) [Z6],
  (0.004217284878422725+0j) [Y6 X7 X8 Y9],
  (-0.003832986432203079+0j) [Y6 Y7 X10 X11],
  (-0.0030498840144275847+0j) [X4 Z5 X6 X7 Z8 Z9 Z10 X11],
  (-0.0009084689845792896+0j) [X0 Z1 Z2 Z3 Z4 Z5 X6 X7 Z8 Z9 Z10 X11],
  (-0.004031650984162859+0j) [X2 Z3 Z4 Z5 X6 X7 Z8 Z9 Z10 X11]],
 6: [(

In [5]:
N_index=0
S_index=0
# molecule_list=['H2', 'LiH', 'BeH2', 'H2O']#, 'CH4']
molecule_list=['LiH']

output=[]
output_multi_count=[]
for Molecule in molecule_list:
    
    
    ### HAMILTONIAN start
    geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
    basis = 'sto-6g'


    ### Get Hamiltonian
    Hamilt = Hamiltonian_PySCF(Molecule,
                         run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                         basis=basis,
                         multiplicity=1,
                         geometry=geometry)  # normally None!
    QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
    ### HAMILTONIAN end
    
    
    ## Graph
    Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

    commutativity_flag = 'AC' ## <- defines relationship between sets!!!
    plot_graph = False
    Graph_colouring_strategy='largest_first'
    anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, 
                                                                            Graph_colouring_strategy=Graph_colouring_strategy,
                                                                            plot_graph=plot_graph)

    
#     ## ansatz
#     ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
    
#     Sec_Quant_CC_ops_ia, Sec_Quant_CC_ops_ijab, theta_parameters_ia, theta_parameters_ijab=ansatz_obj.Get_ia_and_ijab_terms(single_cc_amplitudes=None, 
#                                                                                                              double_cc_amplitudes=None, 
#                                                                                                              singles_hamiltonian=None,
#                                                                                                              doubles_hamiltonian=None, 
#                                                                                                              tol_filter_small_terms = None)

#     Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(
#                                             Sec_Quant_CC_ops_ia,
#                                             Sec_Quant_CC_ops_ijab,
#                                            transformation='JW')
    
    ## UCCSD circuit
    
    
#     input_state=ansatz_obj.Get_JW_HF_state_in_OCC_basis()
#     ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
#                                      Qubit_Op_list_Second_Quant_CC_Ops_ia, 
#                                      Qubit_Op_list_Second_Quant_CC_Ops_ijab)

#     ansatz_circuit =ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                 Theta_param_list_ia=theta_parameters_ia, 
#                                  Theta_param_list_ijab=theta_parameters_ijab,
#                                  ia_first=True)
    ansatz_circuit = cirq.Circuit()
    
    
    ### standard circuit
#     for qubitOP in QubitHamiltonian:
#         if list(qubitOP.terms.keys())[0] != ():
#             Q_Circuit_obj = change_pauliword_to_Z_basis_then_measure(qubitOP)
#             gate_dict, total_gate = Total_Gate_Count(Q_Circuit,
#                                                      M_gates_included=True, 
#                                                      only_one_two_qubit_gates=True)
#             break
    
    ## Standard method is O(number of qubits!)
    total_gate = Hamilt.molecule.n_qubits
    
    
    # find LONGEST ANTI-COMMUTING SET
    max_key, max_value = max(anti_commuting_sets.items(), key = lambda x: len(x[1]))    
    
    
    ### LCU

    R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[max_key], N_index)
    R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1_norm = absorb_complex_phases(R_uncorrected)

    LCU_Q_circuit = Full_Q_Circuit_single_and_two_qubit_gates(Pn,
                                   R_corrected_Op_list,
                                   R_corr_list,
                                   ancilla_amplitudes,
                                   Hamilt.molecule.n_qubits,
                                   ansatz_circuit)
    LCU_gate_dict, LCU_total_gate = Total_Gate_Count(LCU_Q_circuit,
                                             M_gates_included=True, 
                                             only_one_two_qubit_gates=True)

    LCU_total_g_multi_dict, LCU_total_g_multi = Total_Gate_Count(LCU_Q_circuit,
                                             M_gates_included=True, 
                                             only_one_two_qubit_gates=False)
        
    
    ### Conj

    Conj_Q_circuit, Ps, gamma_l = Generate_Full_Q_Circuit_conj(ansatz_circuit,
                                                         anti_commuting_sets[max_key],
                                                         S_index, # <- S_index set to 0
                                                         check_reduction=True)
    
    
    Conj_gate_dict, Conj_total_gate = Total_Gate_Count(Conj_Q_circuit,
                                             M_gates_included=True, 
                                             only_one_two_qubit_gates=True)

    Conj_total_g_multi_dict, Conj_total_g_multi = Total_Gate_Count(Conj_Q_circuit,
                                             M_gates_included=True, 
                                             only_one_two_qubit_gates=False)
            
    
    
    different_approaches_one_two={
                        'standard': {'gates': total_gate,
                                 'repeats': len(list(QubitHamiltonian))
                                    },

                        'LCU': {'gates': LCU_total_gate,
                                 'repeats': len(anti_commuting_sets)
                                    },

                        'Conj': {'gates': Conj_total_gate,
                                 'repeats': len(anti_commuting_sets)
                                    }
                        }
    
    multi_gate ={
                    'LCU_gates': LCU_total_g_multi,
                    'Conj_gates':Conj_total_g_multi
                    }

    output.append(different_approaches_one_two)
    output_multi_count.append(multi_gate)

Building Graph Edges: 100%|##########| 631/631 [00:18<00:00, 34.73it/s] 


In [9]:
LCU_Q_circuit

┌──┐                       ┌──┐                                                               ┌──┐                                                                                   ┌──┐                                                                                                                                                                       ┌──┐                   ┌──┐                                                           ┌──┐                                                                       ┌───────────┐                                                                                                                                                                                                                                                                                                                                                                                                            ┌──┐                   ┌──┐                                                           ┌──┐                                                                                   ┌──┐                   ┌──┐                                                               ┌──┐                   ┌──┐                                                               ┌──┐                   ┌──┐                           ┌──┐                       ┌──┐                           ┌──┐
0: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Y────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              │                                                                                                                                                        │
1: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
output

In [ ]:
output_multi_count

In [ ]:
standard=[]
LCU=[]
Conj=[]
for result in output:
    standard.append(result['standard']['gates'])
    LCU.append(result['LCU']['gates'])
    Conj.append(result['Conj']['gates'])
    
standard_repeats=[]
LCU_repeats=[]
Conj_repeats=[]
for result in output:
    standard_repeats.append(result['standard']['repeats'])
    LCU_repeats.append(result['LCU']['repeats'])
    Conj_repeats.append(result['Conj']['repeats'])

    
multi_LCU=[]
multi_Conj=[]
for result in output_multi_count:
    multi_LCU.append(result['LCU_gates'])
    multi_Conj.append(result['Conj_gates'])

In [ ]:
## import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()

plt.plot(molecule_list, standard, label = "standard",  marker='x', linestyle='-', linewidth=1.5, color='blue')
plt.plot(molecule_list, LCU, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red')
plt.plot(molecule_list, Conj, label = "Conj", marker='x', linestyle='-', linewidth=1.5, color='green')

plt.xlabel('Molecule')
plt.ylabel('total gate count')
plt.legend()
plt.show()


plt.figure()
plt.plot(molecule_list, multi_LCU, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red')
plt.plot(molecule_list, multi_Conj, label = "Conj", marker='o', linestyle='--', linewidth=1.5, color='green')
plt.xlabel('Molecule')
plt.ylabel('total multi gate count')
plt.legend()
plt.show()


plt.figure()
plt.plot(molecule_list, standard_repeats, label = "standard",  marker='x', linestyle='-', linewidth=1.5, color='blue')
plt.plot(molecule_list, LCU_repeats, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red', markersize=12)
plt.plot(molecule_list, Conj_repeats, label = "Conj", marker='o', linestyle='--', linewidth=1.5, color='green')
plt.xlabel('Molecule')
plt.ylabel('No H terms to measure')
plt.legend()
plt.show()



In [ ]:
cirq.Circuit(cirq.decompose(LCU_Q_circuit))

In [ ]:
# https://cirq.readthedocs.io/en/stable/circuits.html
from cirq.circuits import InsertStrategy

In [ ]:
N_index=0
S_index=0
molecule_list=['H2', 'LiH', 'BeH2', 'H2O']#, 'CH4']


output=[]
output_multi_count=[]
for Molecule in molecule_list:
    
    
    ### HAMILTONIAN start
    geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
    basis = 'sto-6g'


    ### Get Hamiltonian
    Hamilt = Hamiltonian_PySCF(Molecule,
                         run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                         basis=basis,
                         multiplicity=1,
                         geometry=geometry)  # normally None!
    QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
    ### HAMILTONIAN end
    
    
    ## Graph
    Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

    commutativity_flag = 'AC' ## <- defines relationship between sets!!!
    plot_graph = False
    Graph_colouring_strategy='largest_first'
    anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, 
                                                                            Graph_colouring_strategy=Graph_colouring_strategy,
                                                                            plot_graph=plot_graph)

    
    ansatz_circuit = cirq.Circuit()
    
    
    ### standard circuit
#     for qubitOP in QubitHamiltonian:
#         if list(qubitOP.terms.keys())[0] != ():
#             Q_Circuit_obj = change_pauliword_to_Z_basis_then_measure(qubitOP)
#             gate_dict, total_gate = Total_Gate_Count(Q_Circuit,
#                                                      M_gates_included=True, 
#                                                      only_one_two_qubit_gates=True)
#             break
    total_gate = Hamilt.molecule.n_qubits #O(N) change of basis gates required!
    
    ### LCU
    for key in anti_commuting_sets:
        if len(anti_commuting_sets[key])>1:
            R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[key], N_index)
            R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1_norm = absorb_complex_phases(R_uncorrected)

            Q_circ_LCU = Full_Q_Circuit(Pn,
                                           R_corrected_Op_list,
                                           R_corr_list,
                                           ancilla_amplitudes,
                                           Hamilt.molecule.n_qubits,
                                           ansatz_circuit)
            
            LCU_Q_circuit = cirq.Circuit(list(Q_circ_LCU.all_operations()), strategy=InsertStrategy.EARLIEST)
            
            LCU_gate_dict, LCU_total_gate = Total_Gate_Count(LCU_Q_circuit,
                                                     M_gates_included=True, 
                                                     only_one_two_qubit_gates=True)
            
            LCU_total_g_multi_dict, LCU_total_g_multi = Total_Gate_Count(LCU_Q_circuit,
                                                     M_gates_included=True, 
                                                     only_one_two_qubit_gates=False)
            break
    
    ### Conj
    for key in anti_commuting_sets:
        if len(anti_commuting_sets[key])>1:
            normalised_set = Get_beta_j_cofactors(anti_commuting_sets[key])
            X_sk_dict = Get_X_sk_operators(normalised_set, S=S_index)
            
            Q_circ_Conj = Generate_Full_Q_Circuit_unitary_part(ansatz_circuit, X_sk_dict)
            Conj_Q_circuit = cirq.Circuit(list(Q_circ_Conj.all_operations()), strategy=InsertStrategy.EARLIEST)
            
            Conj_gate_dict, Conj_total_gate = Total_Gate_Count(Conj_Q_circuit,
                                                     M_gates_included=True, 
                                                     only_one_two_qubit_gates=True)
            
            Conj_total_g_multi_dict, Conj_total_g_multi = Total_Gate_Count(Conj_Q_circuit,
                                                     M_gates_included=True, 
                                                     only_one_two_qubit_gates=False)
            break
    
    
    different_approaches_one_two={
                        'standard': {'gates': total_gate,
                                 'repeats': len(list(QubitHamiltonian))
                                    },

                        'LCU': {'gates': LCU_total_gate,
                                 'repeats': len(anti_commuting_sets)
                                    },

                        'Conj': {'gates': Conj_total_gate,
                                 'repeats': len(anti_commuting_sets)
                                    }
                        }
    
    multi_gate ={
                    'LCU_gates': LCU_total_g_multi,
                    'Conj_gates':Conj_total_g_multi
                    }

    output.append(different_approaches_one_two)
    output_multi_count.append(multi_gate)

In [ ]:
standard=[]
LCU=[]
Conj=[]
for result in output:
    standard.append(result['standard']['gates'])
    LCU.append(result['LCU']['gates'])
    Conj.append(result['Conj']['gates'])
    
standard_repeats=[]
LCU_repeats=[]
Conj_repeats=[]
for result in output:
    standard_repeats.append(result['standard']['repeats'])
    LCU_repeats.append(result['LCU']['repeats'])
    Conj_repeats.append(result['Conj']['repeats'])

    
multi_LCU=[]
multi_Conj=[]
for result in output_multi_count:
    multi_LCU.append(result['LCU_gates'])
    multi_Conj.append(result['Conj_gates'])
    
    
## import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()

plt.plot(molecule_list, standard, label = "standard",  marker='x', linestyle='-', linewidth=1.5, color='blue')
plt.plot(molecule_list, LCU, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red')
plt.plot(molecule_list, Conj, label = "Conj", marker='x', linestyle='-', linewidth=1.5, color='green')

plt.xlabel('Molecule')
plt.ylabel('total gate count')
plt.legend()
plt.show()


plt.figure()
plt.plot(molecule_list, multi_LCU, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red')
plt.plot(molecule_list, multi_Conj, label = "Conj", marker='o', linestyle='--', linewidth=1.5, color='green')
plt.xlabel('Molecule')
plt.ylabel('total multi gate count')
plt.legend()
plt.show()


plt.figure()
plt.plot(molecule_list, standard_repeats, label = "standard",  marker='x', linestyle='-', linewidth=1.5, color='blue')
plt.plot(molecule_list, LCU_repeats, label = "LCU", marker='x', linestyle='-', linewidth=1.5, color='red', markersize=12)
plt.plot(molecule_list, Conj_repeats, label = "Conj", marker='o', linestyle='--', linewidth=1.5, color='green')
plt.xlabel('Molecule')
plt.ylabel('No H terms to measure')
plt.legend()
plt.show()

